## 多输入多输出通道
实现一下多输入通道互相关运算

In [1]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    """
    对每个2D都执行卷积，然后求和
    """
    # X和K都是3D
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

## 验证互相关运算的输出

In [5]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

print(X.shape)
corr2d_multi_in(X, K)

torch.Size([2, 3, 3])


tensor([[ 56.,  72.],
        [104., 120.]])

## 计算多个通道的输出的互相关函数

In [7]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [8]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## $1 \times 1$卷积

In [10]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3,3,3))
K = torch.normal(0, 1, (2, 3, 1,1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out_1x1(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6